We consider the weighted sum of various exponential decay functions with different decay time constants which together give the total envelope. Such envelopes can be useful for extended modal synthesis in which each mode is not only a decaying sinusoid but a sinusoid with a more complex envelope.

In [1]:
# symbolic variables for time, amplitude, attack-time 1, attack-time 2, 
# attack-mix, decay-time 1 decay-time 2, decay-mix:
reset()
var("t A a0 a1 ma d0 d1 md")
att0(t) = exp(-a0*t)  # attack function when ma = 0  
att1(t) = exp(-a1*t)  # attack function when ma = 1
dec0(t) = exp(-d0*t)  # decay  function when md = 0
dec1(t) = exp(-d1*t)  # decay  function when md = 1
att(t) = (1-ma)*att0(t) + ma*att1(t)  # attack function
dec(t) = (1-md)*dec0(t) + md*dec1(t)  # decay  function
env(t) = (dec(t) - att(t))            # envelope

# find position of maximum:
envP(t) = diff(env(t), t)  # env', p: "prime"
assume(a0 >   0)
assume(a1 > a0)
assume(d0 > a1)
assume(d1 > d0)
assume(ma >  0)
assume(ma-1 >  0)
assume(md >  0)
assume(md-1 >  0)
#assume(ma != 0)
eq = envP(t) == 0  
#solve(eq,t)  # doesn't find a solution :-( - maybe 4 is too much - try 3 - only two-stage decay
#energy = integral(env(t)*env(t), t, 0, oo)
area = integral(env(t), t, 0, oo)

env, envP, area

# assign some values to the parameters and plot:
#env1 = env.substitute(a0=1/10,a1=3/10,ma=1/4,d0=2,d1=8,md=1/2)
#plot(-env1(t), 0, 30)

(t |--> (ma - 1)*e^(-a0*t) - ma*e^(-a1*t) - (md - 1)*e^(-d0*t) + md*e^(-d1*t),
 t |--> -a0*(ma - 1)*e^(-a0*t) + a1*ma*e^(-a1*t) + d0*(md - 1)*e^(-d0*t) - d1*md*e^(-d1*t),
 -((a0 - a1)*d0*d1*ma - (a0*a1 - a1*d0)*d1 - (a0*a1*d0 - a0*a1*d1)*md)/(a0*a1*d0*d1))

In [9]:
# for a two-stage decay we mix two decaying exponentials with different
# decay times and adjustable mix/blend factor:
@interact
def twoDecays(Decay = slider(1,   20, 0.1, 10),
              Ratio = slider(0.1,  1, 0.1, 0.1),
              Mix   = slider(0,    1, 0.1, 0.7)):
    d1 = Decay
    d2 = Ratio*Decay
    p = plot( (1-Mix)*exp(-x/d1)+Mix*exp(-x/d2)  , 0, 30)
    p.show()
    

NameError: name 'slider' is not defined

In [9]:
# We try an envelope with one attack and two-stage decay
reset()
var("t a d0 d1 md")
dec0(t) = exp(-d0*t)  # decay  function when md = 0
dec1(t) = exp(-d1*t)  # decay  function when md = 1
dec(t) = (1-md)*dec0(t) + md*dec1(t)  # decay  function
att(t) = exp(-a*t)                    # attack function 
env(t) = (dec(t) - att(t))            # envelope
envP(t) = diff(env(t), t)             # env', p: "prime"
env, envP
#eq = envP(t) == 0  
#solve(eq,t) # also no solution

(t |--> -(md - 1)*e^(-d0*t) + md*e^(-d1*t) - e^(-a*t),
 t |--> d0*(md - 1)*e^(-d0*t) - d1*md*e^(-d1*t) + a*e^(-a*t))

In [8]:
# PK - finally, try just a simple attack/decay shape - this actually has a 
# solution
reset()
var("t a d")
dec(t) = exp(-d*t)         # decay  function
att(t) = exp(-a*t)         # attack function 
env(t) = (dec(t) - att(t)) # envelope
envP(t) = diff(env(t), t)  # env', p: "prime"
env, envP
#eq = envP(t) == 0  
#solve(eq,t) # sage also finds no solution - why?!
# the solution is t = (log(d)-log(a))/(d-a)

(t |--> -e^(-a*t) + e^(-d*t), t |--> a*e^(-a*t) - d*e^(-d*t))

maybe the solution is to fix d0,d1,md,t (where t is the desired time instant of the peak) and then run a Newton iteration
to find the attack parameter a - and maybe this procedure can be extended to two attack-parameters as well, when we define a "major"-attack and a minor-attack = fraction*majorAttack with given fraction. then, fraction is another (fixed) parameter and we solve the newton iteration for the major attack. i think two-stage decay is really desirable and maybe the second attack parameter could translate to some sort of attack-softness (making the shape more sigmoid?)...dunno, but seems reasonable
...but...maybe it is futile to try to adjust the exact location of the envelope peak and we could just give the user the d0,d1,md,a0,a1,ma paremeters directly...the maybe adjust the amplitude according to an energy integral..this seems to be difficult to evaluate due to the square - but the area under the envelope function itself is easy to evaluate - maybe use that

In [5]:
# for a two-stage decay we mix two decaying exponentials with different
# decay times and adjustable mix/blend factor:
@interact
def fourExps(Attack      = slider(0.1,  5, 0.1, 1.5),
             AttackRatio = slider(0.1,  1, 0.1, 0.5),
             AttackMix   = slider(0,    1, 0.1, 0.3),
             Decay       = slider(1,   20, 0.1, 10),
             DecayRatio  = slider(0.1,  1, 0.1, 0.1),
             DecayMix    = slider(0,    1, 0.1, 0.1)):
    d1  = Decay
    d2  = DecayRatio*Decay
    dec = (DecayMix)*exp(-x/d1)+(1-DecayMix)*exp(-x/d2)
    a1  = Attack
    a2  = AttackRatio*Attack
    att = (AttackMix)*exp(-x/a1)+(1-AttackMix)*exp(-x/a2)
    env = dec - att
    p = plot(env, 0, 30)
    #p.show()
# OK - this gives a nice and flexible shape - but sometimes it can go below zero, so taking the area
# for normalization may not be good idea because negative and postive areas may cancel
# maybe the attack time should be defined as a fraction of the decay time

NameError: name 'slider' is not defined

In [4]:
reset()
var("t A B C D a b c d")
f(t) = A*exp(a*t) + B*exp(b*t) - C*exp(c*t) - D*exp(d*t)
e(t) = (f(t))^2
assume(A > 0)
assume(B > 0)
assume(C > 0)
assume(D > 0)
assume(a < 0)
assume(b < 0)
assume(c < 0)
assume(d < 0)
#assume(exp(b+a)-1 > 0)  # integral divergent in this case (takes long to compute)
assume(exp(b+a)-1 < 0)
energy = integral(e(t), t, 0, oo)
f, e, energy
# oookayyy - this gives a verry unwieldy solution but at least, a solution - but it can be optimized
# perhaps, we should use B=1-A, D=1-C

(t |--> A*e^(a*t) + B*e^(b*t) - C*e^(c*t) - D*e^(d*t),
 t |--> (A*e^(a*t) + B*e^(b*t) - C*e^(c*t) - D*e^(d*t))^2,
 -1/2*((D^2*a^3*b^2 + D^2*a^2*b^3)*c^4 + (C^2*a^3*b^2 + C^2*a^2*b^3 + (B^2*a^2 + A^2*b^2 + (A^2 + 4*A*B + B^2)*a*b)*c^3 + (B^2*a^3 + A^2*b^3 + (A^2 + 4*A*B + 2*B^2 - 4*(A + B)*C + C^2)*a^2*b + (2*A^2 + 4*A*B + B^2 - 4*(A + B)*C + C^2)*a*b^2)*c^2 + ((B^2 - 4*B*C + C^2)*a^3*b + (A^2 + 4*A*B + B^2 - 4*(A + B)*C + 2*C^2)*a^2*b^2 + (A^2 - 4*A*C + C^2)*a*b^3)*c)*d^4 + (D^2*a^4*b^2 + 2*D^2*a^3*b^3 + D^2*a^2*b^4)*c^3 + (C^2*a^4*b^2 + 2*C^2*a^3*b^3 + C^2*a^2*b^4 + (B^2*a^2 + A^2*b^2 + (A^2 + 4*A*B + B^2)*a*b)*c^4 + (2*B^2*a^3 + 2*A^2*b^3 + (2*A^2 + 8*A*B + 4*B^2 - 4*(A + B)*C + C^2 - 4*(A + B - C)*D + D^2)*a^2*b + (4*A^2 + 8*A*B + 2*B^2 - 4*(A + B)*C + C^2 - 4*(A + B - C)*D + D^2)*a*b^2)*c^3 + (B^2*a^4 + A^2*b^4 + (A^2 + 4*A*B + 4*B^2 - 4*(A + 2*B)*C + 2*C^2 - 4*(A + B - C)*D + D^2)*a^3*b + 2*(2*A^2 + 6*A*B + 2*B^2 - 6*(A + B)*C + 2*C^2 - 4*(A + B - C)*D + D^2)*a^2*b^2 + (4*A^2 + 4*

In [43]:
reset()
var("t A C a b c d")
f(t) = A*exp(a*t) + (1-A)*exp(b*t) + C*exp(c*t) + (1-C)*exp(d*t)
e(t) = (f(t))^2
assume(A > 0)
assume(1-A > 0)
assume(C > 0)
assume(1-C > 0)
assume(a < 0)
assume(b < 0)
assume(c < 0)
assume(d < 0)
assume(exp(b+a)-1 < 0)
assume(exp(b)-1 < 0)   
assume(exp(d)-1 < 0)
energy = integral(e(t), t, 0, oo)
#energy2 = energy.expand()
f, e, energy
# OK that's better - still quite a moster formula - can it be simplified?

(t |--> A*e^(a*t) - (A - 1)*e^(b*t) + C*e^(c*t) - (C - 1)*e^(d*t),
 t |--> (A*e^(a*t) - (A - 1)*e^(b*t) + C*e^(c*t) - (C - 1)*e^(d*t))^2,
 -1/2*(((C^2 - 2*C + 1)*a^3*b^2 + (C^2 - 2*C + 1)*a^2*b^3)*c^4 + (C^2*a^3*b^2 + C^2*a^2*b^3 + (A^2*b^2 + (A^2 - 2*A + 1)*a^2 - (2*A^2 - 2*A - 1)*a*b)*c^3 + (A^2*b^3 + (A^2 - 2*A + 1)*a^3 - (A^2 - C^2 - 4*C - 2)*a^2*b - (A^2 - C^2 - 2*A - 4*C - 1)*a*b^2)*c^2 + ((A^2 - 4*(A - 1)*C + C^2 - 2*A + 1)*a^3*b - (2*A^2 - 2*C^2 - 2*A - 4*C - 1)*a^2*b^2 + (A^2 + 4*A*C + C^2)*a*b^3)*c)*d^4 + ((C^2 - 2*C + 1)*a^4*b^2 + 2*(C^2 - 2*C + 1)*a^3*b^3 + (C^2 - 2*C + 1)*a^2*b^4)*c^3 + (C^2*a^4*b^2 + 2*C^2*a^3*b^3 + C^2*a^2*b^4 + (A^2*b^2 + (A^2 - 2*A + 1)*a^2 - (2*A^2 - 2*A - 1)*a*b)*c^4 + (2*A^2*b^3 + 2*(A^2 - 2*A + 1)*a^3 - (2*A^2 + 2*C^2 - 2*C - 9)*a^2*b - (2*A^2 + 2*C^2 - 4*A - 2*C - 7)*a*b^2)*c^3 + (A^2*b^4 + (A^2 - 2*A + 1)*a^4 + (A^2 - 2*(2*A - 3)*C - C^2 - 4*A + 9)*a^3*b - 2*(2*A^2 + C^2 - 2*A - 4*C - 7)*a^2*b^2 + (A^2 + 2*(2*A + 1)*C - C^2 + 2*A + 6)*a*b^3)*c^2 